In [1]:
import numpy as np

### Optimizaition Methods
![7](https://user-images.githubusercontent.com/38183218/44017435-440dd208-9f13-11e8-8d7c-0fc626274bc8.PNG)

![1](https://user-images.githubusercontent.com/38183218/44017423-41111d30-9f13-11e8-9181-b582c4973096.PNG)

#### Stochastic Gradient Descent(full batch와 비교)
- 배치마다 gradient 내려가는 모양다르기에 local minima에 빠지는 위험 피할수있음



- 그러나

![1 5](https://user-images.githubusercontent.com/38183218/44017424-413a8de6-9f13-11e8-9117-65d2c48b8d4c.PNG)
![2](https://user-images.githubusercontent.com/38183218/44017425-41632972-9f13-11e8-861d-fc9ad89036b7.PNG)

- SGD의 단점은 비등방성함수(방향에 따라 성질, 즉 기울기가 달라지는 함수)에서 탐색 경로가 비효율적
![](https://user-images.githubusercontent.com/38183218/45130977-e3017f80-b1c5-11e8-8269-27574584cf26.png)
- 예들어 위와같이 y축방향 기울기 가파르지만 x축방향 기울기는 완만한경우. 최적점인 (0,0)에 도달하는 시간이 매우 오래걸림

- 근본적인 한계도 존재, 그레디언트의 방향이 최적점과 다른방향을 가리키는 문제

#### Momentum Update
같은 방향이면 쭉 밀어주기
![3](https://user-images.githubusercontent.com/38183218/44017426-418b4b64-9f13-11e8-92cb-3ec0ca5b61b7.PNG)

In [2]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
            self.lr = lr
            self.momentum = momentum
            self.v = None
            
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zero_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key] 

#### Adagrad Update
노드마다 lr다르게 걸어서 학습률 최적화. 보통 학습률을 서서히 낮추는 형태. 

직관적으로 생각해도 최적점찾으려면 처음에는 보폭을 크게, 나중에는 작게해야.

큰 움직임에 비례해 갱신 정도도 큰폭으로 작아진다.

그래서 SGD와 같이 지그재그 형태로 움직이더라도 y축방향으로 갱신강도가 빠르게 약해지고, 지그재그의 움직임이 줄어든다

cache값이 점점 증가하므로 어느순간 dx가 0 되는 단점 -> RMSProp으로 해결

![4](https://user-images.githubusercontent.com/38183218/44017432-439170aa-9f13-11e8-869d-bddbaea10c9b.PNG)



#### RMSProp Update
기울기를 제곱하여 계속 더해가는 AdaGrad의 cache, 그래서 학습 진행할수록 갱신강도가 약해진다

실제로 어느순간 갱신량이 0이된다

RMSProp은 지수이동평균(EMA)을 사용하여 과거 기울기의 반영 정도를 기하급수적으로 감소시킨다

decay rate를 사용
![5](https://user-images.githubusercontent.com/38183218/44017433-43bab082-9f13-11e8-9fb1-ad410728df20.PNG)


#### Adam Update

Adam Optimization 논문 요약/정리 : http://dalpo0814.tistory.com/29

모멘텀 + RMSprop


In [5]:
# Adam 간단히 구현
m = beta1*m + (1-beta1)*dx 
v = beta2*v + (1-beta2)*(dx**2) 
x -= learning_Rate * m/(np.sqrt(v) + 1e-7)

In [3]:
class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)

![](https://user-images.githubusercontent.com/38183218/45130955-e0068f00-b1c5-11e8-8a4e-cfb1d55eb9cb.png)

### Initial Weights

### Batch Normalization
레이어가 쌓일 수록 이상치인 batch가 가져오는 오차는 커진다

그래서 애초에 zero-centered로 normalize해버리면 어떠냐

: 각 층이 활성화를 적당히 퍼뜨리도록 '강제' 하는 것
- 학습속도개선
- 초기값에 크게 의존x
- 오버피팅 억제해 드롭아웃 등의 필요성 감소

### Regularization
#### Ensembles
#### Add term to loss
#### Dropout
어떤 feature의 가중치가 너무 커지면 drop하여 오버피팅 문제해결
#### Data Augmentation
